In [177]:
import numpy as np
import pandas as pd
import os
import shutil
from checkRF_Score import *

In [178]:
source_folder="dataset/asteroid_dataset/withils/ssim_veryhighmiss_s25_f1000_sites100_GTR_bl1.0_d0.0_l0.0_t0.0_gc0.0_p0.0_pop50000000_ms0.6_mf0.6_seed3016"
families_folder=source_folder+"/families"
gt_file_name="true.true.geneTree.newick"
species_tree_name="speciesTree.newick"

In [179]:
gt_output_folder= "output"
gt_output_file_name="1_gt.tre"
true_species_tree_name= "true-species.out.tree"

In [180]:
!rm -rf $gt_output_folder
!mkdir -p $gt_output_folder

In [181]:
import re

def remove_one(input_str):
    output_str = re.sub(r"1(?=[^\d_])", "", input_str)
    return output_str

def remove_numbers_after_colon(input_str):
    output_str = re.sub(r':[^,\)]+', '', input_str)
    output_str = remove_one(output_str)
    return output_str

def remove_numbers_after_colon_NO_NEED_TO_REMOVE_ONE(input_str):
    output_str = re.sub(r':[^,\)]+', '', input_str)
    return output_str

# input_str = '(22_0_0:1.18959,(((17_0_0:0.0449454,14_0_0:0.0400912)1:0.067281,8_0_0:0.25315)1:0.0472608,12_0_0:0.190684)1:0.409925);'
# output = remove_numbers_after_colon(input_str)
# output = remove_one(output)
# print(input_str)
# print(output)

In [182]:
all_gts = []
for dir in os.listdir(families_folder):
    # print(dir)
    # save mapping from mapping folder
    mapping={}
    mapping_dir="mappings"
    mapping_file_path=families_folder+"/"+dir+"/"+mapping_dir+"/treerecs_mapping.link"
    if os.path.exists(mapping_file_path):
        # print(mapping_file_path)
        with open(mapping_file_path) as f:
            for line in f:
                # print(line)
                (key, val) = line.split()
                mapping[key] = val
    else:
        print("mapping file not found")
    if len(mapping) == 0:
        print("mapping is empty")
    # print(mapping)
    gt_tree_dir="gene_trees"
    gt_tree_file_path=families_folder+"/"+dir+"/"+gt_tree_dir+"/"+gt_file_name
    if not os.path.exists(gt_tree_file_path):
        print("file not exists")
        exit(1)
    with open(gt_tree_file_path) as f:
        line = f.readline()
        line = remove_numbers_after_colon(line)
        # print(line)
        for key in mapping:
            line = line.replace(key, mapping[key])
        all_gts.append(line)
        # print(line)
#output to a file in output folder
with open(gt_output_folder+"/"+gt_output_file_name, "w") as f:
    for gt in all_gts:
        f.write(gt+"\n")

In [183]:
# copy species tree to output folder
species_tree_path = source_folder+"/species_trees/"+species_tree_name
with open(species_tree_path) as f:
    line = f.readline()
    line = remove_numbers_after_colon_NO_NEED_TO_REMOVE_ONE(line)
    with open(gt_output_folder+"/"+true_species_tree_name, "w") as f:
        f.write(line)


In [184]:
true_tree = join_dir(gt_output_folder, true_species_tree_name)

In [185]:
def getTaxaSet(t):
    return t.replace('(', '').replace(')', '').split(';')[0].split(':')[0].split(',')

In [186]:
!rm -rf in/
!rm -rf out/
!mkdir in
!cp output/1_gt.tre in/
!python3 postprocessor.py
!cp out/complete/astral/1_gt_species.out.tre output/input.nwk


================== ASTRAL ===================== 

This is ASTRAL version 5.6.3
Gene trees are treated as unrooted
507 trees read from /Users/sayemhasan/Desktop/Thesis/Terraces/raxml-ng/in/1_gt.tre
All output trees will be *arbitrarily* rooted at 23

======== Running the main analysis
Number of taxa: 25 (25 species)
Taxa: [23, 21, 24, 20, 11, 13, 14, 7, 15, 22, 3, 12, 8, 19, 17, 1, 9, 5, 18, 2, 10, 4, 6, 25, 16]
Taxon occupancy: {22=303, 23=138, 24=146, 25=42, 10=25, 11=101, 12=209, 13=191, 14=184, 15=197, 16=14, 17=130, 18=112, 19=47, 1=49, 2=63, 3=178, 4=41, 5=163, 6=61, 7=91, 8=141, 9=194, 20=67, 21=103}
Number of gene trees: 507
507 trees have missing taxa
Calculating quartet distance matrix (for completion of X)
Species tree distances calculated ...
Will attempt to complete bipartitions from X before adding using a distance matrix.
Building set of clusters (X) from gene trees 
------------------------------
gradient0: 225
Number of Clusters after addition by distance: 225
calculat

In [187]:
OUTGROUP = 'GAL'

st = open('output/input.nwk')
st = st.read()
gts = open('output/1_gt.tre')

gts = gts.read().split(';\n')[:-1] 

# print(len(gts))

# print(st)
taxaSet = getTaxaSet(st)
# print(taxaSet)
matrix = np.zeros((len(taxaSet), len(gts)), dtype=int)


# print(len(gts))
for gt_idx, gt in  enumerate(gts) :
    # print(gt)
    cur_taxaSet = getTaxaSet(gt)
    # print(cur_taxaSet)
    for i, tx in enumerate(taxaSet):
        if tx  in cur_taxaSet or tx == OUTGROUP:
            matrix[i][gt_idx] = 1

In [188]:
with open('output/input.data', 'w', newline='') as f:
    print(matrix.shape[0], matrix.shape[1], file=f)
    for i in range(len(taxaSet)):
        # f.write(matrix[i].tostring(), taxaSet[i])
        print(*matrix[i],taxaSet[i], end='\n', file=f)

In [189]:
!rm -rf terrace_output
!mkdir -p terrace_output
!./bin/raxml-ng output/input.nwk output/input.data > terrace_output/terrace_output.txt

In [190]:

#get RF score of ASTRAL OUTPUT
csv_out_list = []
astral_rf_score = calculate_RF_distance_for_a_ST(join_dir(gt_output_folder, "input.nwk"), true_tree)
print("astral_rf_score", astral_rf_score)
csv_out_list += [(ASTRAL,INCOMPLETE ,"" , "", astral_rf_score)]

#get RF score for each tree in Terrace
terrace_rf_scores=calculate_RF_distance_for_terrace(join_dir(TERRACE_OUTPUT_FOLDER, "terrace_output.txt"), true_tree)
# print("terrace_rf_scores", terrace_rf_scores)
if terrace_rf_scores is not None:
    csv_out_list+=terrace_rf_scores




astral_rf_score 0.22727272727272727


In [191]:
# get consensus tree
terrace_dir = join_dir(TERRACE_OUTPUT_FOLDER)
consensus_dir = join_dir(terrace_dir, "consensus")
!rm -rf "$consensus_dir"
!mkdir "$consensus_dir"

if not dir_exists_abs(terrace_dir):
    print("Directory does not exist: {}".format(terrace_dir))
    exit(1)


In [192]:
for terrace_file in os.listdir(terrace_dir):
    file_path = join_dir(terrace_dir, terrace_file)
    if file_path.endswith(".txt"):
        print("Processing file: {}".format(file_path))
        terrace_file_name = terrace_file.split(".")[0]
        !./bin/raxml-ng --consense MRE --tree "$file_path" --prefix "$consensus_dir"/"$terrace_file_name"

Processing file: terrace_output/terrace_output.txt

RAxML-NG v. 1.1.0-master released on 04.08.2022 by The Exelixis Lab.
Developed by: Alexey M. Kozlov and Alexandros Stamatakis.
Contributors: Diego Darriba, Tomas Flouri, Benoit Morel, Sarah Lutteropp, Ben Bettisworth.
Latest version: https://github.com/amkozlov/raxml-ng
Questions/problems/suggestions? Please visit: https://groups.google.com/forum/#!forum/raxml

System: Apple M1, 8 cores, 8 GB RAM

RAxML-NG was called at 30-Jan-2023 20:11:36 as follows:

./bin/raxml-ng --consense MRE --tree terrace_output/terrace_output.txt --prefix terrace_output/consensus/terrace_output

Analysis options:
  run mode: Build consensus tree (MRE)
  start tree(s): user
  random seed: 1675087896
  SIMD kernels: SSE3
  parallelization: coarse-grained (auto), PTHREADS (auto)

Reading input trees from file: terrace_output/terrace_output.txt
Loaded 3 trees with 25 taxa.

MRE consensus tree saved to: /Users/sayemhasan/Desktop/Thesis/Terraces/raxml-ng/terrace_o

In [193]:
def remove_numbers_from_consensus_tree(file_path):
    file = open(file_path, "r")
    lines = file.readlines()
    line = lines[0]
    line = re.sub(r'[0-9]', '', line)
    # print(line)
    file = open(file_path, 'w')
    file.writelines(line)
    file.close()

In [194]:
# print("Removing numbers from consensus trees")
CONSENSUS_TREE_EXTENSION = "consensusTreeMRE"
for consensus_file in os.listdir(consensus_dir):
    if consensus_file.endswith(CONSENSUS_TREE_EXTENSION):
        file_path = join_dir(consensus_dir, consensus_file)
        print("Processing file: {}".format(file_path))
        # remove_numbers_from_consensus_tree(file_path)
        dist = calculate_RF_distance_for_a_ST(file_path, true_tree)
        print("Consensus RF distance: {}".format(dist))
        csv_out_list.append(["ASTRAL", "Consensus","","", dist])
print("Astral RF Score : ", astral_rf_score)

stat_fl = 'stats/RF.csv'
with open(stat_fl, 'w') as stat_file:
    csv.writer(stat_file).writerows(csv_out_list)

Processing file: terrace_output/consensus/terrace_output.raxml.consensusTreeMRE
Consensus RF distance: 0.2727272727272727
Astral RF Score :  0.22727272727272727
